## Setup environment variables


In [ ]:
from getpass import getpass

# Database connection parameters
DB_HOST = input("Enter the database host: ")
DB_NAME = "merge"
DB_USER = "postgres"
DB_PASS = getpass("Enter the database password: ")
DB_PORT = "5432"

In [ ]:
# Path to your GADM GeoPackage file (All in one layer version)
GADM_FILE = input("Enter the path to your GADM GeoPackage file: ")


## (Optional) SQL to create schema, tables, indexes, and views


In [ ]:
-- Enable PostGIS extension if not already enabled
CREATE EXTENSION IF NOT EXISTS postgis;

-- Create table for admin level 0 (country level)
CREATE TABLE gadm_admin0 (
    gid_0 VARCHAR(3) PRIMARY KEY,
    name_0 VARCHAR(100) NOT NULL,
    geometry GEOMETRY(MultiPolygon, 4326)
);

-- Create table for admin level 1 (typically state/province level)
CREATE TABLE gadm_admin1 (
    gid_1 VARCHAR(10) PRIMARY KEY,
    gid_0 VARCHAR(3) REFERENCES gadm_admin0(gid_0),
    name_1 VARCHAR(100) NOT NULL,
    type_1 VARCHAR(50),
    engtype_1 VARCHAR(50),
    geometry GEOMETRY(MultiPolygon, 4326)
);

-- Create table for admin level 2 (typically county/district level)
CREATE TABLE gadm_admin2 (
    gid_2 VARCHAR(15) PRIMARY KEY,
    gid_1 VARCHAR(10) REFERENCES gadm_admin1(gid_1),
    gid_0 VARCHAR(3) REFERENCES gadm_admin0(gid_0),
    name_2 VARCHAR(100) NOT NULL,
    type_2 VARCHAR(50),
    engtype_2 VARCHAR(50),
    geometry GEOMETRY(MultiPolygon, 4326)
);

-- Create spatial indexes for faster querying
CREATE INDEX idx_gadm_admin0_geometry ON gadm_admin0 USING GIST(geometry);
CREATE INDEX idx_gadm_admin1_geometry ON gadm_admin1 USING GIST(geometry);
CREATE INDEX idx_gadm_admin2_geometry ON gadm_admin2 USING GIST(geometry);

-- Create regular indexes for common lookup fields
CREATE INDEX idx_gadm_admin1_gid_0 ON gadm_admin1(gid_0);
CREATE INDEX idx_gadm_admin2_gid_1 ON gadm_admin2(gid_1);
CREATE INDEX idx_gadm_admin2_gid_0 ON gadm_admin2(gid_0);

-- Create a view that combines all admin levels
CREATE OR REPLACE VIEW gadm_combined AS
SELECT 
    gid_0 AS gid,
    name_0 AS name,
    NULL AS parent_gid,
    0 AS admin_level,
    'Country' AS level_type,
    'Country' AS level_engtype,
    geometry
FROM 
    gadm_admin0

UNION ALL

SELECT 
    gid_1 AS gid,
    name_1 AS name,
    gid_0 AS parent_gid,
    1 AS admin_level,
    COALESCE(type_1, 'Admin Level 1') AS level_type,
    COALESCE(engtype_1, 'Admin Level 1') AS level_engtype,
    geometry
FROM 
    gadm_admin1

UNION ALL

SELECT 
    gid_2 AS gid,
    name_2 AS name,
    gid_1 AS parent_gid,
    2 AS admin_level,
    COALESCE(type_2, 'Admin Level 2') AS level_type,
    COALESCE(engtype_2, 'Admin Level 2') AS level_engtype,
    geometry
FROM 
    gadm_admin2;

-- Create indexes on the view for better performance
CREATE INDEX idx_gadm_combined_gid ON gadm_combined(gid);
CREATE INDEX idx_gadm_combined_admin_level ON gadm_combined(admin_level);
CREATE INDEX idx_gadm_combined_parent_gid ON gadm_combined(parent_gid);
CREATE INDEX idx_gadm_combined_level_type ON gadm_combined(level_type);
CREATE INDEX idx_gadm_combined_level_engtype ON gadm_combined(level_engtype);


## Python to run the SQL to create the schema, tables, indexes, and view


In [ ]:
# %pip install sqlalchemy psycopg2-binary


In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Create the database connection string
db_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create a SQLAlchemy engine
engine = create_engine(db_url)

# Define the schema for MERGE
merge_schema = """
-- Create gadm_admin0 table
CREATE TABLE IF NOT EXISTS gadm_admin0 (
    ISO3 CHAR(3) PRIMARY KEY,
    admin_level_0 VARCHAR(100) NOT NULL,
    geometry GEOMETRY(MULTIPOLYGON, 4326)
);

-- Create gadm_admin1 table
CREATE TABLE IF NOT EXISTS gadm_admin1 (
    ISO3 CHAR(3),
    GID_1 VARCHAR(20) PRIMARY KEY,
    ISO3_1 VARCHAR(20),
    admin_level_1 VARCHAR(100) NOT NULL,
    admin_level_1_var_name TEXT[],
    admin_level_1_nl_name TEXT[],
    geometry GEOMETRY(MULTIPOLYGON, 4326),
    FOREIGN KEY (ISO3) REFERENCES gadm_admin0(ISO3)
);

-- Create gadm_admin2 table
CREATE TABLE IF NOT EXISTS gadm_admin2 (
    ISO3 CHAR(3),
    GID_1 VARCHAR(20),
    GID_2 VARCHAR(20) PRIMARY KEY,
    admin_level_2 VARCHAR(100) NOT NULL,
    admin_level_2_var_name TEXT[],
    admin_level_2_nl_name TEXT[],
    geometry GEOMETRY(MULTIPOLYGON, 4326),
    FOREIGN KEY (ISO3) REFERENCES gadm_admin0(ISO3),
    FOREIGN KEY (GID_1) REFERENCES gadm_admin1(GID_1)
);

-- Create spatial indexes
CREATE INDEX IF NOT EXISTS idx_gadm_admin0_geometry ON gadm_admin0 USING GIST(geometry);
CREATE INDEX IF NOT EXISTS idx_gadm_admin1_geometry ON gadm_admin1 USING GIST(geometry);
CREATE INDEX IF NOT EXISTS idx_gadm_admin2_geometry ON gadm_admin2 USING GIST(geometry);
"""


def create_merge_schema():
    with engine.connect() as connection:
        try:
            # Enable PostGIS extension if not already enabled
            connection.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))

            # Create the new schema
            connection.execute(text(merge_schema))

            connection.commit()
            print("MERGE schema created successfully.")
        except SQLAlchemyError as e:
            print(
                f"An error occurred while creating the MERGE schema: {str(e)}"
            )
            connection.rollback()


create_merge_schema()


## Python to transform GADM data to PostGIS in a PostgreSQL database


In [ ]:
# %pip install geopandas sqlalchemy psycopg2-binary geoalchemy2


In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from geoalchemy2 import Geometry
from shapely import wkb

# Create the database connection string
db_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create a SQLAlchemy engine
engine = create_engine(db_url)


def validate_and_clean_geometries(gdf):
    """Validate and clean geometries in the GeoDataFrame."""
    # Check for null geometries
    null_geoms = gdf[gdf["geometry"].isnull()]
    if not null_geoms.empty:
        print(
            f"Warning: {len(null_geoms)} rows with null geometries found. These will be removed."
        )
        gdf = gdf.dropna(subset=["geometry"])

    # Check for valid geometries
    invalid_geoms = gdf[~gdf["geometry"].is_valid]
    if not invalid_geoms.empty:
        print(
            f"Warning: {len(invalid_geoms)} rows with invalid geometries found. Attempting to fix..."
        )
        gdf.loc[~gdf["geometry"].is_valid, "geometry"] = gdf.loc[
            ~gdf["geometry"].is_valid, "geometry"
        ].buffer(0)
        still_invalid = gdf[~gdf["geometry"].is_valid]
        if not still_invalid.empty:
            print(
                f"  {len(still_invalid)} geometries could not be fixed and will be removed."
            )
            gdf = gdf[gdf["geometry"].is_valid]

    return gdf


def geometry_to_wkb(geom):
    """Convert geometry to WKB format."""
    if geom is None:
        return None
    return wkb.dumps(geom, hex=True)


def import_gadm_data():
    """Import GADM data into the new schema."""
    # Read the GeoPackage file
    gdf = gpd.read_file(GADM_FILE)
    print(f"Original dataset size: {len(gdf)} rows")

    # Validate and clean geometries
    gdf = validate_and_clean_geometries(gdf)
    print(f"Dataset size after cleaning: {len(gdf)} rows")

    # Ensure the CRS is set to EPSG:4326 (WGS84)
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
        print("Data reprojected to EPSG:4326 (WGS84)")

    # Convert geometries to WKB format and then to hex, handling potential null values
    gdf["geometry"] = gdf["geometry"].apply(geometry_to_wkb)

    # Remove any rows where geometry conversion failed (resulting in None)
    gdf = gdf.dropna(subset=["geometry"])
    print(f"Final dataset size: {len(gdf)} rows")

    # Connect to the database
    with engine.connect() as connection:
        # Import admin level 0 data
        admin0_data = gdf[["GID_0", "NAME_0", "geometry"]].drop_duplicates(
            subset=["GID_0"]
        )
        admin0_data.columns = ["iso3", "admin_level_0", "geometry"]
        admin0_data.to_sql(
            "gadm_admin0",
            connection,
            if_exists="append",
            index=False,
            dtype={"geometry": Geometry("MULTIPOLYGON", srid=4326)},
        )
        print(
            f"Admin level 0 data imported successfully. {len(admin0_data)} rows inserted."
        )

        # Import admin level 1 data
        admin1_data = gdf[
            [
                "GID_0",
                "GID_1",
                "ISO_1",
                "NAME_1",
                "VARNAME_1",
                "NL_NAME_1",
                "geometry",
            ]
        ].drop_duplicates(subset=["GID_1"])
        admin1_data.columns = [
            "iso3",
            "gid_1",
            "iso3_1",
            "admin_level_1",
            "admin_level_1_var_name",
            "admin_level_1_nl_name",
            "geometry",
        ]
        # Convert var_name and nl_name to arrays, handling null, single, and multiple values
        admin1_data["admin_level_1_var_name"] = admin1_data[
            "admin_level_1_var_name"
        ].apply(lambda x: x.split("|") if x else None)
        admin1_data["admin_level_1_nl_name"] = admin1_data[
            "admin_level_1_nl_name"
        ].apply(lambda x: x.split("|") if x else None)
        admin1_data.to_sql(
            "gadm_admin1",
            connection,
            if_exists="append",
            index=False,
            dtype={"geometry": Geometry("MULTIPOLYGON", srid=4326)},
        )
        print(
            f"Admin level 1 data imported successfully. {len(admin1_data)} rows inserted."
        )

        # Import admin level 2 data
        admin2_data = gdf[
            [
                "GID_0",
                "GID_1",
                "GID_2",
                "NAME_2",
                "VARNAME_2",
                "NL_NAME_2",
                "geometry",
            ]
        ].drop_duplicates(subset=["GID_2"])
        admin2_data.columns = [
            "iso3",
            "gid_1",
            "gid_2",
            "admin_level_2",
            "admin_level_2_var_name",
            "admin_level_2_nl_name",
            "geometry",
        ]
        # Convert var_name and nl_name to arrays, handling null, single, and multiple values
        admin2_data["admin_level_2_var_name"] = admin2_data[
            "admin_level_2_var_name"
        ].apply(lambda x: x.split("|") if x else None)
        admin2_data["admin_level_2_nl_name"] = admin2_data[
            "admin_level_2_nl_name"
        ].apply(lambda x: x.split("|") if x else None)
        admin2_data.to_sql(
            "gadm_admin2",
            connection,
            if_exists="append",
            index=False,
            dtype={"geometry": Geometry("MULTIPOLYGON", srid=4326)},
        )
        print(
            f"Admin level 2 data imported successfully. {len(admin2_data)} rows inserted."
        )


try:
    import_gadm_data()
    print("GADM data import completed successfully.")
except SQLAlchemyError as e:
    print(f"An error occurred while importing GADM data: {str(e)}")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")
